In [ ]:
import pandas as pd
import numpy as np
import geopandas as gp
from datetime import datetime
from datetime import date
import calendar
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from multiprocessing import Pool
# from haversine import haversine
from textwrap import wrap
import os.path
import json
import pickle
import zipfile
import seaborn as sns
from functools import partial
import matplotlib.collections as mcoll
import matplotlib.path as mpath
# import contextily as ctx
import math

from matplotlib import animation
from matplotlib.animation import FuncAnimation, PillowWriter 

%matplotlib inline

In [ ]:
matplotlib.rcParams.update({
    'figure.dpi': 80,
    'figure.titlesize': 25,
    'axes.titlesize': 15,
    'axes.labelsize': 13,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 13,
    'legend.title_fontsize': 15
})

# WLRA

In [ ]:
day = 1
df_nyc = pd.read_csv("result/nyc_ssa_"+str(day)+".csv")
df_chicago = pd.read_csv("result/chicago_ssa_"+str(day)+".csv")

In [ ]:
fig, axs = plt.subplots(1,2,figsize = (7,3),sharex=True,sharey=True)


ind = 0

lss = ['-','--','-.',':']

day = 1

df_nyc = pd.read_csv("result/nyc_ssa_"+str(day)+".csv")
df_chicago = pd.read_csv("result/chicago_ssa_"+str(day)+".csv")

axs[0].plot(range(1,51),df_nyc['MSE.fhv'][1:51]/df_nyc['MSE.fhv'][0], ls = lss[0], label = 'NYC FHV')
axs[0].plot(range(1,51),df_nyc['MSE.taxi'][1:51]/df_nyc['MSE.taxi'][0], ls = lss[0], label = 'NYC Taxi')
axs[0].plot(range(1,51),df_nyc['MSE.bike'][1:51]/df_nyc['MSE.bike'][0], ls = lss[0], label = 'NYC BSS')

axs[0].plot(range(1,51),df_chicago['MSE.fhv'][1:51]/df_chicago['MSE.fhv'][0], ls = lss[1], label = 'Chicago FHV')
axs[0].plot(range(1,51),df_chicago['MSE.taxi'][1:51]/df_chicago['MSE.taxi'][0], ls = lss[1], label = 'Chicago Taxi')
axs[0].plot(range(1,51),df_chicago['MSE.bike'][1:51]/df_chicago['MSE.bike'][0], ls = lss[1], label = 'Chicago BSS')

axs[1].plot(range(1,51),df_nyc['MAE.fhv'][1:51]/df_nyc['MAE.fhv'][0], ls = lss[0], label = 'NYC FHV')
axs[1].plot(range(1,51),df_nyc['MAE.taxi'][1:51]/df_nyc['MAE.taxi'][0], ls = lss[0], label = 'NYC Taxi')
axs[1].plot(range(1,51),df_nyc['MAE.bike'][1:51]/df_nyc['MAE.bike'][0], ls = lss[0], label = 'NYC BSS')

axs[1].plot(range(1,51),df_chicago['MAE.fhv'][1:51]/df_chicago['MAE.fhv'][0], ls = lss[1], label = 'Chicago FHV')
axs[1].plot(range(1,51),df_chicago['MAE.taxi'][1:51]/df_chicago['MAE.taxi'][0], ls = lss[1], label = 'Chicago Taxi')
axs[1].plot(range(1,51),df_chicago['MAE.bike'][1:51]/df_chicago['MAE.bike'][0], ls = lss[1], label = 'Chicago BSS')

axs[0].set_ylabel("Relative error")
# axs[1].set_ylabel("Relative reconstruction")
# axs[0][1].legend()
# axs[1][0].legend()
# axs[1][1].legend()
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

axs[0].set_xticks(range(0,51,5))
axs[1].set_xticks(range(0,51,5))

axs[0].grid()
axs[1].grid()

axs[0].set_xlabel("Number of signals")
axs[1].set_xlabel("Number of signals")

axs[0].set_ylim([0,.5])
axs[1].set_ylim([0,.5])

axs[0].set_xlim([0,30])
axs[1].set_xlim([0,30])

axs[0].set_title("MSE")
axs[1].set_title("MAE")

# titles = ['NYC FHV','NYC Taxi','NYC BSS','Chicago FHV','Chicago Taxi','Chicago BSS']

# for i in range(6):
#     axs[0][i].set_title(titles[i])
#     axs[1][i].set_xlabel("Number of signals")
# axs[1][1].set_xlabel("Number of signals")

plt.tight_layout()

plt.savefig('fig/ssa.png', dpi = 300, format = 'png')
plt.savefig('fig/ssa.pdf', dpi = 300, format = 'pdf')

In [ ]:
df_nyc_true = pd.read_csv("result/nyc_true.csv").values[:,1:]
df_chicago_true = pd.read_csv("result/chicago_true.csv").values[:,1:]

In [ ]:
fig, ax = plt.subplots(1,1)

x1 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_nyc_true))]
colors = sns.color_palette('Paired', n_colors = df_nyc_true.shape[1])
for i in range(df_nyc_true.shape[1]):
    ax.plot(x1, df_nyc_true[:,i],color = colors[i],lw= 1,alpha=0.8)
plt.xlim([datetime(2019, 1, 1), datetime(2021, 7, 31)])
plt.ylim([0,40])
plt.tick_params(axis='x', rotation=30)
plt.yticks([])
plt.xticks([])
plt.savefig('fig/overview1.png', dpi = 300, format = 'png')
plt.savefig('fig/overview1.pdf', dpi = 300, format = 'pdf')

In [ ]:
fig, ax = plt.subplots(1,1)

df_nyc = pd.read_csv("result/nyc_signal_"+str(day)+".csv").values[:,1:]
x1 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_nyc_true))]
colors = sns.color_palette('Paired', n_colors = df_nyc.shape[1])

ax.plot(x1, -df_nyc[:,0]*3,color = colors[0],lw= 1,alpha=0.8)
for i in range(1,5):
    ax.plot(x1, df_nyc[:,i]-0.15*i,color = colors[i],lw= 1,alpha=0.8)
plt.xlim([datetime(2019, 1, 1), datetime(2021, 7, 31)])
# plt.ylim([0,40])
plt.tick_params(axis='x', rotation=30)
plt.yticks([])
plt.xticks([])
plt.savefig('fig/overview2.png', dpi = 300, format = 'png')
plt.savefig('fig/overview2.pdf', dpi = 300, format = 'pdf')

In [ ]:
df_nyc_true = pd.read_csv("result/nyc_true.csv").values[:,1:]
df_chicago_true = pd.read_csv("result/chicago_true.csv").values[:,1:]
df_nyc_reconstruct = pd.read_csv("result/nyc_reconstruct.csv").values[:,1:]
df_chicago_reconstruct = pd.read_csv("result/chicago_reconstruct.csv").values[:,1:]

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (6,3),sharex=True,sharey=True)


x_ind = np.random.randint(len(df_nyc_true), size = 200)
y_ind = np.random.randint(142,size = 200) + 142*0
axs[0].scatter(df_nyc_true[x_ind, y_ind],df_nyc_reconstruct[x_ind, y_ind], marker = '+', color = 'C2', label = 'FHV', s=40, alpha=0.8,zorder = 2)
x_ind = np.random.randint(len(df_nyc_true), size = 200)
y_ind = np.random.randint(142,size = 200) + 142*1
axs[0].scatter(df_nyc_true[x_ind, y_ind],df_nyc_reconstruct[x_ind, y_ind], marker = 'x',color = 'C1', label = 'Taxi', s=20, alpha=0.8,zorder = 2)
x_ind = np.random.randint(len(df_nyc_true), size = 200)
y_ind = np.random.randint(142,size = 200) + 142*2
axs[0].scatter(df_nyc_true[x_ind, y_ind],df_nyc_reconstruct[x_ind, y_ind], marker = '*',color = 'C3', label = 'BSS', s=30, alpha=0.8,zorder = 2)

x = df_nyc_true.flatten()
y = df_nyc_reconstruct.flatten()
axs[0].hist2d(x,y,bins=(np.arange(0,50,.5),np.arange(0,50,.5)), cmap = plt.cm.Blues, cmin=0, vmax=10, zorder=1 )

x_ind = np.random.randint(len(df_chicago_true), size = 200)
y_ind = np.random.randint(154,size = 200) + 154*2
axs[1].scatter(df_chicago_true[x_ind, y_ind],df_chicago_reconstruct[x_ind, y_ind], marker = '+', color = 'C2', label = 'FHV', s=40, alpha=0.8, zorder = 2)
x_ind = np.random.randint(len(df_chicago_true), size = 200)
y_ind = np.random.randint(154,size = 200) + 154*2
axs[1].scatter(df_chicago_true[x_ind, y_ind],df_chicago_reconstruct[x_ind, y_ind], marker = 'x', color = 'C1', label = 'Taxi', s=20, alpha=0.8, zorder = 2)
x_ind = np.random.randint(len(df_chicago_true), size = 200)
y_ind = np.random.randint(154,size = 200) + 154*2
axs[1].scatter(df_chicago_true[x_ind, y_ind],df_chicago_reconstruct[x_ind, y_ind], marker = '*',color = 'C3', label = 'BSS', s=30, alpha=0.8, zorder = 2)

x = df_chicago_true.flatten()
y = df_chicago_reconstruct.flatten()
axs[1].hist2d(x,y,bins=(np.arange(0,50,.5),np.arange(0,50,.5)), cmap = plt.cm.Blues, cmin=0, vmax=10, zorder=1 )


for i in range(2):
    axs[i].plot([-10,60],[-10,60],ls='--',color = 'black', lw=1)
    axs[i].set_xticks(range(0,51,10))
    axs[i].set_xlim([0,50])
    axs[i].set_ylim([0,50])
    axs[i].grid()
    axs[i].set_xlabel("Original data")
axs[0].set_ylabel("Reconstructed data")

axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

axs[0].set_title('NYC')
axs[1].set_title('Chicago')

plt.tight_layout()

plt.savefig('fig/ssa2.png', dpi = 300, format = 'png')
plt.savefig('fig/ssa2.pdf', dpi = 300, format = 'pdf')

In [ ]:
fig, ax = plt.subplots(1, figsize = (6,7),sharex=True)

day = 1

df_nyc = pd.read_csv("result/nyc_signal_"+str(day)+".csv")
df_chicago = pd.read_csv("result/chicago_signal_"+str(day)+".csv")

x1 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_nyc))]
x2 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_chicago))]

# Decide which is which 
# for ind in range(1,11):
#     res = adfuller(df_nyc['V'+str(ind)])
#     stat, p  = res[0], res[1]
#     if(p>0.1 and ind):
#         nyc_groups[0].append(ind)
#     elif(is_seasonal(df_nyc['V'+str(ind)].values)):
#         nyc_groups[1].append(ind)
#     else:
#         nyc_groups[2].append(ind)
#     res = adfuller(df_chicago['V'+str(ind)])
#     stat, p  = res[0], res[1]
#     if(p>0.1 and ind):
#         chicago_groups[0].append(ind)
#     elif(is_seasonal(df_chicago['V'+str(ind)].values)):
#         chicago_groups[1].append(ind)
#     else:
#         chicago_groups[2].append(ind)
    

ind = 1
for j in range(10):
    nyc_shift= 0.4 * j
    chicago_shift = 0.15+nyc_shift
    
    if((ind<=1)):
        ax.plot(x1,-3*df_nyc['V'+str(ind)]-nyc_shift, lw=1, color = 'C0')
    elif(ind==2):
        ax.plot(x1,2*df_nyc['V'+str(ind)]-nyc_shift, lw=1, color = 'C0')
    else:
        ax.plot(x1,df_nyc['V'+str(ind)]-nyc_shift, lw=1, color = 'C0')
    if((ind<=1)):
        ax.plot(x2,-3*df_chicago['V'+str(ind)]-chicago_shift,  lw= .5, color = 'C3')
    elif(ind==2):
        ax.plot(x2,2*df_chicago['V'+str(ind)]-chicago_shift,  lw=.5, color = 'C3')
    elif(ind==6 or ind == 7 or ind==9):
        ax.plot(x2,-df_chicago['V'+str(ind)]-chicago_shift,  lw=.5, color = 'C3')
    else:
        ax.plot(x2,df_chicago['V'+str(ind)]-chicago_shift,  lw=.5, color = 'C3')
    ax.text(x=datetime(2018, 7, 1),y=-nyc_shift,s='Signal '+str(ind),fontsize=11)
    ind += 1

ax.set_title("Mobility signals")

ax.set_yticks([])
# for j in range(3):
#     axs[j].set_yticks([])

ax.plot([],[], alpha= 1,lw=1, color = 'C0', label='NYC')
ax.plot([],[], alpha= 1,lw=.5, color = 'C3', label='Chicago')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# axs[0].text(x=datetime(2019, 1, 1) + timedelta(50),y=0.06+0.15,s='Signal 1')
# axs[0].text(x=datetime(2019, 1, 1) + timedelta(650),y=0.06+0.15,s='Signal 2')

# axs[0].text(x=datetime(2019, 1, 1) + timedelta(50),y=0.06,s='Signal 1')
# axs[0].text(x=datetime(2019, 1, 1) + timedelta(650),y=0.06,s='Signal 2')

# axs[2].text(x=datetime(2019, 1, 1) + timedelta(50),y=0.2,s='Signal '+ ','.join([str(i) for i in nyc_groups[2]]))
# axs[2].text(x=datetime(2019, 1, 1) + timedelta(50),y=-0.17,s='Signal '+ ','.join([str(i) for i in chicago_groups[2]]))

ax.xaxis.set_major_locator(matplotlib.dates.MonthLocator(interval=3))
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%b'%y"))
ax.tick_params(axis='x', rotation=30)
# axs[1].tick_params(axis='x', rotation=30)
# axs[2].tick_params(axis='x', rotation=30)

ax.set_xlim([datetime(2019, 1, 1), datetime(2021, 7, 31)])
ax.set_ylim([-4,.25])

# ax.vlines(x = datetime(2020,3,22), ymin=-5,ymax=1,color='black')
# ax.text(x=datetime(2020,3,24), y = 0.1, s='Stay-at-home started',fontsize=11)

# axs[0].text(x=50,y=0.08,s='Signal 1')

# axs[0].text(x=50,y=0.05,s='Signal 3')
# axs[0].text(x=10,y=-0.03,s='Signal 2')
# axs[0].text(x=40,y=-0.08,s='Signal 1')
plt.tight_layout()

plt.savefig('fig/ssa3.png', dpi = 300, format = 'png')
plt.savefig('fig/ssa3.pdf', dpi = 300, format = 'pdf')

# Model selection

In [ ]:
from datetime import timedelta
from datetime import datetime

subs = ['_20','_52','_20_1','_52_new']
sub = subs[1]
df_nyc = pd.read_csv("result/nyc_one_step_MAE"+sub+".csv")
df_chicago = pd.read_csv("result/chicago_one_step_MAE"+sub+".csv")

x1 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_nyc))]
x2 = [datetime(2019, 1, 1) + timedelta(i) for i in range(len(df_chicago))]

df_nyc = df_nyc.iloc[:,1:].values
df_chicago = df_chicago.iloc[:,1:].values

In [ ]:
fig, axs = plt.subplots(2,3,figsize = (12,6),sharex=True)
plt.subplots_adjust(wspace = 0.2, hspace= .2)

axs[0][0].set_title("FHV")
axs[0][1].set_title("Taxi")
axs[0][2].set_title("BSS")

# axs[1][0].set_title("Chicago FHV")
# axs[1][1].set_title("Chicago Taxi")
# axs[1][2].set_title("Chicago BSS")

colors = sns.color_palette("Set1", 12)
colors = colors[:4]+colors[8:]
# names = ["No predictor","$x_{weather}$","$x_{COVID}$", "All predictors"]
names = ["No predictor","$X_{weather}$","$X_{COVID}$","$X_{policy}$","$X_{weather}+X_{COVID}$","$X_{weather}+X_{policy}$","$X_{COVID}+X_{policy}$","All predictors"]
ls = ["-","--","-.",":","--","-.",":","-"]

for i in range(3):
    k = 0
    for j in [0,1,2,3,4,5,6,7]:
        alpha = 1
        
        if j == 7:
            alpha = 0.6
        axs[0][i].plot(x1,np.cumsum(np.abs(df_nyc[0:,i+j*3])), color = colors[k], label = names[k], linestyle = ls[k],alpha = alpha)
        axs[1][i].plot(x2,np.cumsum(np.abs(df_chicago[0:,i+j*3])), color = colors[k], label = names[k], linestyle = ls[k],alpha = alpha)
        k+=1

axs[0][2].legend(loc='center left', bbox_to_anchor=(1, 0.2),ncol = 1)

axs[1][0].xaxis.set_major_locator(matplotlib.dates.MonthLocator(interval=3))
axs[1][0].xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%b'%y"))
axs[1][0].tick_params(axis='x', rotation=30)
axs[1][1].tick_params(axis='x', rotation=30)
axs[1][2].tick_params(axis='x', rotation=30)

axs[1][0].set_xlim([datetime(2019, 1, 1), datetime(2021, 7, 31)])
# axs[0].text(x=50,y=0.08,s='Signal 1')

axs[0][0].set_ylabel("NYC cumulative MAE")
axs[1][0].set_ylabel("Chicago cumulative MAE")

for i in range(3):
    axs[0][i].grid()
    axs[1][i].grid()
    axs[0][i].vlines(x = x1[81-15+365], ymin=0, ymax = 800, color = 'C3')
    axs[1][i].vlines(x = x1[80-12+365], ymin=0, ymax = 800, color = 'C3')

# axs[0][0].set_ylim([0,350])
# axs[0][1].set_ylim([0,450])
# axs[0][0].text(x=x1[90], y = 210, s='State of \nemergency',fontsize=11)
# axs[0][1].text(x=x1[90], y = 250, s='State of \nemergency',fontsize=11)
# axs[1][0].set_ylim([0,200])
# axs[1][1].set_ylim([0,300])
# axs[1][0].text(x=x1[90], y = 130, s='State of \nemergency',fontsize=11)
# axs[1][1].text(x=x1[90], y = 200, s='State of \nemergency',fontsize=11)
for j in range(2):
    axs[j][0].set_ylim([0,400])
    axs[j][1].set_ylim([0,500])
    axs[j][2].set_ylim([0,800])
    axs[j][0].text(x=x1[90], y = 250, s='State of \nemergency',fontsize=11)
    axs[j][1].text(x=x1[90], y = 320, s='State of \nemergency',fontsize=11)
    axs[j][2].text(x=x1[90], y = 480, s='State of \nemergency',fontsize=11)
    
#     axs[j][1].set_yticks(range(0, 601, 100))
    
plt.tight_layout()
# plt.show()
plt.savefig('fig/one_step_error.png', dpi = 300, format = 'png')
plt.savefig('fig/one_step_error.pdf', dpi = 300, format = 'pdf')

# BSTS results

In [ ]:
nyc_policies = ['State of emergency', 'Pool rides suspension', 'PAUSE', 'Free bus ride', 'Reduced schedule', 'Mask mandate',
               'Subway nightly closure', 'One-week curfew', 'Phase 1 reopen', 'Phase 2 reopen', 'Phase 3 reopen', 'Phase 4 reopen']

chicago_policies = ['Disaster prolamation', 'Pool rides suspension', 'Stay-at-home', 'Discount for bikesharing', 'Liquor sales curfew', 'Mask mandate',
               'Phase 3 reopen', 'Phase 4 reopen', 'None-essential Business curfew']

In [ ]:
nyc_order = ['(Intercept)', 'RAIN','RAIN_LOG', 'TMAX', 'TMIN', 'CASE', 'VAC']

for i in range(1,11):
    nyc_order.append('S'+str(i))
    nyc_order.append('L'+str(i))
    nyc_order.append('E'+str(i))
    
    
chicago_order = ['(Intercept)', 'RAIN', 'RAIN_LOG', 'TMAX', 'TMIN', 'CASE', 'VAC']

for i in range(1,10):
    chicago_order.append('S'+str(i)) # start linear
    chicago_order.append('L'+str(i)) # incremental linear
    chicago_order.append('E'+str(i)) # end exp

In [ ]:
# Variable coefficients
nyc_coef = pd.read_csv("result/coef_nyc.csv")
chicago_coef = pd.read_csv("result/coef_chicago.csv")

In [ ]:
nyc_coef['signal_id'] = nyc_coef.index//len(nyc_order)
chicago_coef['signal_id'] = chicago_coef.index//len(chicago_order)

nyc_coef['fac_id'] = nyc_coef['Unnamed: 0'].apply(lambda x: nyc_order.index(x))
chicago_coef['fac_id'] = chicago_coef['Unnamed: 0'].apply(lambda x: chicago_order.index(x))

In [ ]:
nyc_coef = nyc_coef.sort_values(by=['signal_id','fac_id'])#.groupby(['signal_id']).head(16)
chicago_coef = chicago_coef.sort_values(by=['signal_id','fac_id'])#.groupby(['signal_id']).head(14)

In [ ]:
nyc_names = ['PH1','MR1','PH2','MR2','RO1','RO2','RO3','RO4','MR3','BR1']
chicago_names = ['PH1','MR1','BR1','PH2','RO1','RO2','BR2','MR2','RO3']

In [ ]:
def highlight_cell(x,y, ax=None, **kwargs):
    rect = plt.Rectangle((x-.5, y-1.5), 1,1, fill=False, **kwargs)
    ax = ax or plt.gca()
    ax.add_patch(rect)
    return rect

In [ ]:
chicago_u_mean = pd.read_csv("result/chicago_u_mean.csv").fillna(0)
nyc_u_mean = pd.read_csv("result/nyc_u_mean.csv").fillna(0)
chicago_u_mean['VAC'] += chicago_u_mean['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

In [ ]:
other_names = ['RAIN', 'RAIN_LOG', 'TMAX', 'TMIN', 'CASE', 'VAC']
other_names2 = ['RAIN','RAIN_LOG', 'TMAX', 'TMIN', 'CASE',  'VAC']
other_labels = ['RAIN','log(RAIN)','TMAX','TRNG','CASE','VAC']

In [ ]:
for i in range(6):
    for k in range(10):
        if(other_names2[i] in chicago_u_mean.columns):
#             if(chicago_coef['inc.prob'].values.reshape(10,36).T[1:9,:][i][k]>0.1):
            if(chicago_u_mean.loc[k][other_names2[i]]!=0):
                if(k==0):
                    print(other_names2[i])

In [ ]:
fig, axs = plt.subplots(2,4, figsize = (12, 6), sharex = True)#, sharey='row')
plt.subplots_adjust(wspace = 0.25, hspace= 0.2)

pres = ['S','L','E']


axs[0][0].imshow(nyc_coef['inc.prob'].values.reshape(10,37).T[1:7,:],cmap='Reds', vmax = 1, vmin = 0)
axs[1][0].imshow(chicago_coef['inc.prob'].values.reshape(10,34).T[1:7,:],cmap='Reds', vmax = 1, vmin = 0)
axs[0][0].set_aspect('auto')
axs[1][0].set_aspect('auto')

axs[1][0].set_yticks([i-0.5 for i in range(6)])
axs[1][0].set_yticklabels([])
axs[1][0].set_yticks([i for i in range(6)], minor=True)
axs[1][0].set_yticklabels(other_labels, minor = True)
axs[1][0].tick_params(which='minor', length=0)

axs[0][0].set_yticks([i-0.5 for i in range(6)])
axs[0][0].set_yticklabels([])
axs[0][0].set_yticks([i for i in range(6)], minor=True)
axs[0][0].set_yticklabels(other_labels, minor = True)
axs[0][0].tick_params(which='minor', length=0)

axs[0][0].set_xticks([i+.5 for i in range(10)])
axs[0][0].set_xticklabels([])
axs[0][0].set_xticks([i for i in range(10)], minor=True)
axs[0][0].set_xticklabels(range(1,11), minor=True)

axs[0][0].set_xlim([-.5,9.5])

axs[1][0].set_xticks([i+.5 for i in range(10)])
axs[1][0].set_xticklabels([])
axs[1][0].set_xticks([i for i in range(10)], minor=True)
axs[1][0].set_xticklabels(range(1,11), minor=True)
axs[1][0].set_xlim([-.5,9.5])

axs[0][0].grid(color = 'white', lw=2, zorder = 3)
axs[1][0].grid(color = 'white', lw=2, zorder = 3)


for i in range(6):
    for k in range(10):
        if(other_names2[i] in chicago_u_mean.columns):
#             if(chicago_coef['inc.prob'].values.reshape(10,36).T[1:9,:][i][k]>0.1):
            if(chicago_u_mean.loc[k][other_names2[i]]!=0):
                highlight_cell(k,i+1, ax = axs[1][0],color="limegreen", linewidth=2.5,zorder=2)

for i in range(6):
    for k in range(10):
        if(other_names[i] in nyc_u_mean.columns):
#             if(nyc_coef['inc.prob'].values.reshape(10,39).T[1:9,:][i][k]>0.1):
            if(nyc_u_mean.loc[k][other_names[i]]!=0):
                highlight_cell(k,i+1, ax = axs[0][0],color="limegreen", linewidth=2.5,zorder=2)

for j in range(3):
#     nyc_colors = ['limegreen' if x>=0 else 'cyan' for x in nyc_signs[np.arange(4+j, 34, 3).astype("int")]]
#     chicago_colors = ['limegreen' if x>=0 else 'cyan' for x in chicago_signs[np.arange(4+j, 31, 3).astype("int")]]
    axs[0][j+1].imshow(nyc_coef['inc.prob'].values.reshape(10, 37).T[1:,][np.arange(6+j, 36, 3).astype("int"),:],cmap='Reds', vmax = 1, vmin = 0)
    axs[0][j+1].set_aspect('auto')
    
    im2  = axs[1][j+1].imshow(chicago_coef['inc.prob'].values.reshape(10, 34).T[1:,][np.arange(6+j, 33, 3).astype("int"),:],cmap='Reds', vmax = 1, vmin = 0)
    axs[1][j+1].set_aspect('auto')
    
    axs[1][j+1].set_yticks([i-0.5 for i in range(9)])
    axs[1][j+1].set_yticklabels([])
    axs[1][j+1].set_yticks([i for i in range(9)], minor=True)
    axs[1][j+1].set_yticklabels(chicago_names, minor = True)
    axs[1][j+1].tick_params(which='minor', length=0)

    axs[0][j+1].set_yticks([i-0.5 for i in range(10)])
    axs[0][j+1].set_yticklabels([])
    axs[0][j+1].set_yticks([i for i in range(10)], minor=True)
    axs[0][j+1].set_yticklabels(nyc_names, minor = True)
    axs[0][j+1].tick_params(which='minor', length=0)

    axs[0][j+1].set_xticks([i+.5 for i in range(10)])
    axs[0][j+1].set_xticklabels([])
    axs[0][j+1].set_xticks([i for i in range(10)], minor=True)
    axs[0][j+1].set_xticklabels(range(1,11), minor=True)

    axs[0][j+1].set_xlim([-.5,9.5])

    axs[1][j+1].set_xticks([i+.5 for i in range(10)])
    axs[1][j+1].set_xticklabels([])
    axs[1][j+1].set_xticks([i for i in range(10)], minor=True)
    axs[1][j+1].set_xticklabels(range(1,11), minor=True)
    axs[1][j+1].set_xlim([-.5,9.5])

    axs[0][j+1].grid(color = 'white', lw=2, zorder = 3)
    axs[1][j+1].grid(color = 'white', lw=2, zorder = 3)
    
# add grid here
    for i in range(9):
        for k in range(10):
            if(str(pres[j])+str(i+1) in chicago_u_mean.columns):
#                  if(chicago_coef['inc.prob'].values.reshape(10, 36).T[1:,][np.arange(8+j, 35, 3).astype("int"),:][i][k]>0.1):
                if(chicago_u_mean.loc[k][str(pres[j])+str(i+1)]!=0):
                    highlight_cell(k,i+1, ax = axs[1][j+1],color="limegreen", linewidth=2.5,zorder=2)

    for i in range(10):
        for k in range(10):
            if(str(pres[j])+str(i+1) in nyc_u_mean.columns):
#                 if(nyc_coef['inc.prob'].values.reshape(10, 39).T[1:,][np.arange(8+j, 38, 3).astype("int"),:][i][k]>0.1):
                if(nyc_u_mean.loc[k][str(pres[j])+str(i+1)]!=0):
                    highlight_cell(k,i+1, ax = axs[0][j+1],color="limegreen", linewidth=2.5,zorder=2)
    
axs[0][0].set_ylabel("NYC", fontsize=16)
axs[1][0].set_ylabel("Chicago", fontsize=16)
# axs[0][2].set_title("NYC Type3")
# axs[0][3].set_title("NYC Other")

axs[0][0+1].set_title("$x_{begin}$")
axs[0][1+1].set_title("$x_{linear}$")
axs[0][2+1].set_title("$x_{end}$")
axs[0][0].set_title("$x_{Weather} & x_{COVID} $")

axs[1][0+1].set_xlabel("Mobility signal")
axs[1][1+1].set_xlabel("Mobility signal")
axs[1][2+1].set_xlabel("Mobility signal")
axs[1][0].set_xlabel("Mobility signal")


cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])

fig.colorbar(im2, cax=cbar_ax)
cbar_ax.get_yaxis().labelpad = 15
cbar_ax.set_ylabel('Inclusion probability', rotation=270, fontsize=14)

plt.savefig('fig/new_coef2.png', dpi = 300, format = 'png')
plt.savefig('fig/new_coef2.pdf', dpi = 300, format = 'pdf')

In [ ]:
nyc_slope = list(1/np.array([512.0, 497.0, 471.0, 426.0, 419.0 ,405.0 ,391.0, 377.0 ,299.0, 230.0 ,
                             275.0, 470.0, 236.0, 423.0, 412.5, 398.5, 384.5, 189.0, 249.5, 199.0 ,
                             37.0, 442.0,   0.0 ,  0.0 ,  0.0,   0.0,   0.0 ,  0.0, 199.0, 167.0]))

chicago_slope = list(1/np.array([510.0, 498.0 ,450.0, 457.0 ,424.0, 403.0 ,282.0, 258.0 , 85.0 ,
                             255.5, 461.5, 262.5, 229.0 ,414.0, 244.5, 232.0, 224.5,  43.0 ,  
                             0.0 ,  0.0,  74.0 ,  0.0 ,  0.0 ,  0.0 ,181.0, 190.0 ,  0.0]))

In [ ]:
nyc_slope2 = list(1/np.array([475 , 55, 471, 7,  14,  14,  14, 377, 100,  63, 230]))
chicago_slope2 = list(1/np.array([510,  74, 376, 457,  21, 318, 101,  68, 85, 229]))

In [ ]:
# Variable coefficients
nyc_coef = pd.read_csv("result/coef_nyc.csv")
chicago_coef = pd.read_csv("result/coef_chicago.csv")

In [ ]:
nyc_names = pd.read_csv("result/nyc_u_mean.csv").columns[1:]
chicago_names = pd.read_csv("result/chicago_u_mean.csv").columns[1:]

nyc_multiplier = []
chicago_multiplier = []

nyc_multiplier2 = []
chicago_multiplier2 = []

i=0
for name in nyc_names:
    if(name.startswith('L')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[10+ind-1]*nyc_slope2[ind-1])
    elif(name.startswith('S')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[ind-1])
    elif(name.startswith('E')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[20+ind-1])
    elif(name.startswith("RAIN")):
        nyc_multiplier.append(1/347)
    elif(name.startswith("TM")):
        nyc_multiplier.append(1/943)
    elif(name.startswith("CASE")):
        nyc_multiplier.append(1/442)
    elif(name.startswith("VAC")):
        nyc_multiplier.append(1/230)
    i+=1

i=0
for name in chicago_names:
    if(name.startswith('L')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[9+ind-1]*chicago_slope2[ind-1])
    elif(name.startswith('S')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[ind-1])
    elif(name.startswith('E')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[18+ind-1])
    elif(name.startswith("RAIN")):
        chicago_multiplier.append(1/340)
    elif(name.startswith("TM")):
        chicago_multiplier.append(1/943)
    elif(name.startswith("CASE")):
        chicago_multiplier.append(1/514)
    elif(name.startswith("VAC")):
        chicago_multiplier.append(1/229)
    else:
        chicago_multiplier.append(1)
    i+=1
        
nyc_multiplier = np.array(nyc_multiplier)*943
chicago_multiplier = np.array(chicago_multiplier)*943

In [ ]:
nyc_num = 142
chicago_num = 154
nyc_v = pd.read_csv("result/nyc_v.csv").values[:,1:]
chicago_v = pd.read_csv("result/chicago_v.csv").values[:,1:]


# contrib, row: factors + signals, col: mean impact, 25 percentile and 75 percentile
chicago_u_mean = pd.read_csv("result/chicago_u_mean.csv").fillna(0)
nyc_u_mean = pd.read_csv("result/nyc_u_mean.csv").fillna(0)
chicago_u_mean['VAC'] += chicago_u_mean['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

nyc_contrib = nyc_u_mean.values[:,1:]
chicago_contrib = chicago_u_mean.values[:,1:]
nyc_impact = nyc_contrib.T @ nyc_v.T
chicago_impact = chicago_contrib.T @ chicago_v.T
# nyc_impact = np.nan_to_num(nyc_impact.astype('float'))
# chicago_impact = np.nan_to_num(chicago_impact.astype('float'))
nyc_res_mean = []
chicago_res_mean = []
for h in range(3):
    nyc_res_mean.append(np.sum(nyc_impact[:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
    chicago_res_mean.append(np.sum(chicago_impact[:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
nyc_res_mean = np.stack(nyc_res_mean)
chicago_res_mean = np.stack(chicago_res_mean)

# contrib, row: factors + signals, col: mean impact, 25 percentile and 75 percentile
chicago_u_median = pd.read_csv("result/chicago_u_median.csv").fillna(0)
nyc_u_median = pd.read_csv("result/nyc_u_median.csv").fillna(0)
chicago_u_median['VAC'] += chicago_u_median['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

nyc_contrib = nyc_u_median.values[:,1:]
chicago_contrib = chicago_u_median.values[:,1:]
nyc_impact = nyc_contrib.T @ nyc_v.T
chicago_impact = chicago_contrib.T @ chicago_v.T
nyc_impact = np.nan_to_num(nyc_impact.astype('float'))
chicago_impact = np.nan_to_num(chicago_impact.astype('float'))
nyc_res_median = []
chicago_res_median = []
for h in range(3):
    nyc_res_median.append(np.sum(nyc_impact[:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
    chicago_res_median.append(np.sum(chicago_impact[:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
nyc_res_median = np.stack(nyc_res_median)
chicago_res_median = np.stack(chicago_res_median)

# 
chicago_u_upper = pd.read_csv("result/chicago_u_75.csv").fillna(0)
nyc_u_upper = pd.read_csv("result/nyc_u_75.csv").fillna(0)
chicago_u_upper['VAC'] += chicago_u_upper['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']
chicago_u_lower = pd.read_csv("result/chicago_u_25.csv").fillna(0)
nyc_u_lower = pd.read_csv("result/nyc_u_25.csv").fillna(0)
chicago_u_lower['VAC'] += chicago_u_lower['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

nyc_contrib_upper = nyc_u_upper.values[:,1:]
chicago_contrib_upper = chicago_u_upper.values[:,1:]
nyc_contrib_lower = nyc_u_lower.values[:,1:]
chicago_contrib_lower = chicago_u_lower.values[:,1:]

nyc_impact_upper = []
nyc_impact_lower = []
chicago_impact_upper = []
chicago_impact_lower = []
for k in range(10):
    nyc_impact_upper.append(nyc_contrib_upper[k,:][:,None] @ nyc_v[:,k][:,None].T)
    nyc_impact_lower.append(nyc_contrib_lower[k,:][:,None] @ nyc_v[:,k][:,None].T)
    chicago_impact_upper.append(chicago_contrib_upper[k,:][:,None] @ chicago_v[:,k][:,None].T)
    chicago_impact_lower.append(chicago_contrib_lower[k,:][:,None] @ chicago_v[:,k][:,None].T)

nyc_res_upper = []
chicago_res_upper = []
nyc_res_lower = []
chicago_res_lower = []
for k in range(10):
    nyc_res_upper_tmp = []
    chicago_res_upper_tmp = []
    nyc_res_lower_tmp = []
    chicago_res_lower_tmp = []
    for h in range(3):
        nyc_res_upper_tmp.append(np.sum(nyc_impact_upper[k][:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
        chicago_res_upper_tmp.append(np.sum(chicago_impact_upper[k][:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
        nyc_res_lower_tmp.append(np.sum(nyc_impact_lower[k][:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
        chicago_res_lower_tmp.append(np.sum(chicago_impact_lower[k][:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
    nyc_res_upper_tmp = np.stack(nyc_res_upper_tmp)
    chicago_res_upper_tmp = np.stack(chicago_res_upper_tmp)
    nyc_res_lower_tmp = np.stack(nyc_res_lower_tmp)
    chicago_res_lower_tmp = np.stack(chicago_res_lower_tmp)
    nyc_res_upper.append(nyc_res_upper_tmp)
    chicago_res_upper.append(chicago_res_upper_tmp)
    nyc_res_lower.append(nyc_res_lower_tmp)
    chicago_res_lower.append(chicago_res_lower_tmp)
    
nyc_res_upper_ = np.sum(np.maximum(nyc_res_upper, nyc_res_lower),axis = 0)
nyc_res_lower_ = np.sum(np.minimum(nyc_res_upper, nyc_res_lower),axis = 0)
chicago_res_upper_ = np.sum(np.maximum(chicago_res_upper, chicago_res_lower),axis = 0)
chicago_res_lower_ = np.sum(np.minimum(chicago_res_upper, chicago_res_lower),axis = 0)

nyc_res_75 = nyc_res_upper_
nyc_res_25 = nyc_res_lower_
chicago_res_75 = chicago_res_upper_
chicago_res_25 = chicago_res_lower_               
# 
chicago_u_upper = pd.read_csv("result/chicago_u_95.csv").fillna(0)
nyc_u_upper = pd.read_csv("result/nyc_u_95.csv").fillna(0)
chicago_u_upper['VAC'] += chicago_u_upper['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']
chicago_u_lower = pd.read_csv("result/chicago_u_5.csv").fillna(0)
nyc_u_lower = pd.read_csv("result/nyc_u_5.csv").fillna(0)
chicago_u_lower['VAC'] += chicago_u_lower['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

nyc_contrib_upper = nyc_u_upper.values[:,1:]
chicago_contrib_upper = chicago_u_upper.values[:,1:]
nyc_contrib_lower = nyc_u_lower.values[:,1:]
chicago_contrib_lower = chicago_u_lower.values[:,1:]

nyc_impact_upper = []
nyc_impact_lower = []
chicago_impact_upper = []
chicago_impact_lower = []
for k in range(10):
    nyc_impact_upper.append(nyc_contrib_upper[k,:][:,None] @ nyc_v[:,k][:,None].T)
    nyc_impact_lower.append(nyc_contrib_lower[k,:][:,None] @ nyc_v[:,k][:,None].T)
    chicago_impact_upper.append(chicago_contrib_upper[k,:][:,None] @ chicago_v[:,k][:,None].T)
    chicago_impact_lower.append(chicago_contrib_lower[k,:][:,None] @ chicago_v[:,k][:,None].T)

nyc_res_upper = []
chicago_res_upper = []
nyc_res_lower = []
chicago_res_lower = []
for k in range(10):
    nyc_res_upper_tmp = []
    chicago_res_upper_tmp = []
    nyc_res_lower_tmp = []
    chicago_res_lower_tmp = []
    for h in range(3):
        nyc_res_upper_tmp.append(np.sum(nyc_impact_upper[k][:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
        chicago_res_upper_tmp.append(np.sum(chicago_impact_upper[k][:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
        nyc_res_lower_tmp.append(np.sum(nyc_impact_lower[k][:,nyc_num*h:(nyc_num*(h+1))], axis = 1))
        chicago_res_lower_tmp.append(np.sum(chicago_impact_lower[k][:,chicago_num*h:(chicago_num*(h+1))], axis = 1))
    nyc_res_upper_tmp = np.stack(nyc_res_upper_tmp)
    chicago_res_upper_tmp = np.stack(chicago_res_upper_tmp)
    nyc_res_lower_tmp = np.stack(nyc_res_lower_tmp)
    chicago_res_lower_tmp = np.stack(chicago_res_lower_tmp)
    nyc_res_upper.append(nyc_res_upper_tmp)
    chicago_res_upper.append(chicago_res_upper_tmp)
    nyc_res_lower.append(nyc_res_lower_tmp)
    chicago_res_lower.append(chicago_res_lower_tmp)
    
nyc_res_upper_ = np.sum(np.maximum(nyc_res_upper, nyc_res_lower),axis = 0)
nyc_res_lower_ = np.sum(np.minimum(nyc_res_upper, nyc_res_lower),axis = 0)
chicago_res_upper_ = np.sum(np.maximum(chicago_res_upper, chicago_res_lower),axis = 0)
chicago_res_lower_ = np.sum(np.minimum(chicago_res_upper, chicago_res_lower),axis = 0)

nyc_res_95 = nyc_res_upper_
nyc_res_5 = nyc_res_lower_
chicago_res_95 = chicago_res_upper_
chicago_res_5 = chicago_res_lower_   

nyc_res_95 = np.nan_to_num(nyc_res_95.astype(float))*nyc_multiplier[None,:]
nyc_res_5 = np.nan_to_num(nyc_res_5.astype(float))*nyc_multiplier[None,:]
chicago_res_95 = np.nan_to_num(chicago_res_95.astype(float))*chicago_multiplier[None,:]
chicago_res_5 = np.nan_to_num(chicago_res_5.astype(float))*chicago_multiplier[None,:]
nyc_res_75 = np.nan_to_num(nyc_res_75.astype(float))*nyc_multiplier[None,:]
nyc_res_25 = np.nan_to_num(nyc_res_25.astype(float))*nyc_multiplier[None,:]
chicago_res_75 = np.nan_to_num(chicago_res_75.astype(float))*chicago_multiplier[None,:]
chicago_res_25 = np.nan_to_num(chicago_res_25.astype(float))*chicago_multiplier[None,:]
nyc_res_median = np.nan_to_num(nyc_res_median.astype(float))*nyc_multiplier[None,:]
chicago_res_median = np.nan_to_num(chicago_res_median.astype(float))*chicago_multiplier[None,:]
nyc_res_mean = np.nan_to_num(nyc_res_mean.astype(float))*nyc_multiplier[None,:]
chicago_res_mean = np.nan_to_num(chicago_res_mean.astype(float))*chicago_multiplier[None,:]

In [ ]:
nyc_fhv_mean, nyc_taxi_mean, nyc_bike_mean = 9842.222*142, 3440.607*142, 747.0655*142
chicago_fhv_mean, chicago_taxi_mean, chicago_bike_mean = 3712.273*154, 534.7464*154, 133.4529*154

nyc_mean = [nyc_fhv_mean, nyc_taxi_mean, nyc_bike_mean]
chicago_mean = [chicago_fhv_mean, chicago_taxi_mean, chicago_bike_mean]

In [ ]:
import matplotlib.patches as mpatches

from matplotlib.patches import Patch

# nyc_days1 = 347 # number of rainy days
# nyc_days2 = [int(1/i) for i in nyc_slope]

# chicago_days1 = 340 # number of rainy days
# chicago_days2 = [int(1/i) for i in chicago_slope]

def sort_box(boxes, names, labels, days):
    res_boxes = []
    for name in names:
        i = 0
        for box in boxes:
            if box['name'] == name:
                if(len(box['name'])<=3 and box['name']!='VAC'):
                    ind = int(box['name'][1:])-1
                    if(box['label']!=''):
                        box['label'] = labels[ind] + "_" + box['label'][0] if box['label'][0]!='L' else labels[ind] + '\n({0})'.format(days[i])
                else:
                    if(box['label']!=''):
                        box['label'] += '\n({0})'.format(days[i])
                res_boxes.append(box)
            i+=1
        
    return res_boxes

other_labels = ['RAIN','RAIN_LOG','TMAX','TMIN','CASE','VAC']
nyc_type0 = ['RAIN','RAIN_LOG', 'TMAX','TMIN', 'CASE', 'VAC'] 
nyc_type1 = ['S'+str(i+1) for i in range(11)] +  ['E'+str(i+1) for i in range(11)]
nyc_type2 =  ['L'+str(i+1) for i in range(11)]
chicago_type0 = ['RAIN','RAIN_LOG', 'TMAX','TMIN', 'CASE', 'VAC'] 
chicago_type1 = ['S'+str(i+1) for i in range(10)] +  ['E'+str(i+1) for i in range(10)]
chicago_type2 = ['L'+str(i+1) for i in range(10)]
nyc_labels = ['PH1','MR1','PH2','MR2','RO1','RO2','RO3','RO4','MR3','BR1','VAC']
chicago_labels = ['PH1','MR1','BR1','PH2','RO1','RO2','BR2','MR2','RO3','VAC']

In [ ]:
fig,axs = plt.subplots(2,3, figsize= (14,12),sharex='col')
colors = ['C0','C1','C2']
current_color = 'w'

xlims = [[-50,120], [-70,70],[-1.2,1.5]]

for k in range(3):
    boxes = []
    boxes2 = []
    boxes3 = []
    days = []
    days2 = []
    days3 = []
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(nyc_res_mean.shape[1]):
        if (nyc_names[i] in nyc_type0):
            i1+=1
            w1 = nyc_res_5[k, i]/nyc_mean[k]
            q1 = nyc_res_25[k, i]/nyc_mean[k]
            q3 = nyc_res_75[k, i]/nyc_mean[k]
            w3 = nyc_res_95[k, i]/nyc_mean[k]
            mean = nyc_res_mean[k, i]/nyc_mean[k]
            med = nyc_res_median[k, i]/nyc_mean[k]
#             if((nyc_names[i]=='TMAX') and k==2):
#                 w1-=0.15
#                 q1-=0.15
#                 q3-=0.15
#                 w3-=0.15
#                 mean-=0.15
#                 med-=0.15
            boxes.append({
                'label' : nyc_names[i] if k==1 else '',
                'name': nyc_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i1%2!=0):
                axs[0][0].add_patch(
                   mpatches.Rectangle((xlims[0][0],i1-.5), xlims[0][1]-xlims[0][0],1, color='lightgrey')
                )
            if(nyc_names[i]=='L11'):
                days.append(230)
            else:
                days.append(round(943/nyc_multiplier[i]))
        elif(nyc_names[i].startswith("L")):
            i3+=1
            w1 = nyc_res_5[k, i]/nyc_mean[k]
            q1 = nyc_res_25[k, i]/nyc_mean[k]
            q3 = nyc_res_75[k, i]/nyc_mean[k]
            w3 = nyc_res_95[k, i]/nyc_mean[k]
            mean = nyc_res_mean[k, i]/nyc_mean[k]
            med = nyc_res_median[k, i]/nyc_mean[k]
#             if(chicago_names[i]=='L2' and k==2):
#                 w1/=2
#                 q1/=2
#                 q3/=2
#                 w3/=2
#                 mean/=2
#                 med/=2
#                 w1-=0.01
#                 q1-=0.01
#                 q3-=0.01
#                 w3-=0.01
#                 mean-=0.01
#                 med-=0.01
            boxes3.append({
                'label' : nyc_names[i] if k==1 else '',
                'name': nyc_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i3%2!=0):
                axs[0][2].add_patch(
                   mpatches.Rectangle((xlims[2][0],i3-.5), xlims[2][1]-xlims[2][0],1, color='lightgrey')
                )
            days3.append(round(1/nyc_slope2[int(nyc_names[i][1:])-1]))
        else:
            i2+=1
            w1 = nyc_res_5[k, i]/nyc_mean[k]
            q1 = nyc_res_25[k, i]/nyc_mean[k]
            q3 = nyc_res_75[k, i]/nyc_mean[k]
            w3 = nyc_res_95[k, i]/nyc_mean[k]
            mean = nyc_res_mean[k, i]/nyc_mean[k]
            med = nyc_res_median[k, i]/nyc_mean[k]
            boxes2.append({
                'label' : nyc_names[i] if k==1 else '',
                'name': nyc_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i2%2!=0):
                axs[0][1].add_patch(
                   mpatches.Rectangle((xlims[1][0],i2-.5), xlims[1][1]-xlims[1][0],1, color='lightgrey')
                )
            days2.append(round(943/nyc_multiplier[i]))
    
    
    boxes = sort_box(boxes, nyc_type0, other_labels, days)
    boxes2 = sort_box(boxes2, nyc_type1, nyc_labels, days2)
    boxes3 = sort_box(boxes3, nyc_type2, nyc_labels, days3)
    
    x = np.arange(len(boxes)) + (k-1)*0.3
    axs[0][0].bxp(boxes,widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[0][0].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[0][0].set_ylim([-.45,len(boxes)-1+0.45])
    axs[0][0].invert_yaxis()
    
    x = np.arange(len(boxes2)) + (k-1)*0.3
    axs[0][1].bxp(boxes2, widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[0][1].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[0][1].set_ylim([-.45,len(boxes2)-1+0.45])
    axs[0][1].invert_yaxis()
    
    print(len(boxes3))
    x = np.arange(len(boxes3)) + (k-1)*0.3
    axs[0][2].bxp(boxes3, widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[0][2].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[0][2].set_ylim([-.45,len(boxes3)-1+0.45])
    axs[0][2].invert_yaxis()
    
    boxes = []
    boxes2 = []
    boxes3 = []
    days = []
    days2 = []
    days3 = []
    i1 = 0
    i2 = 0
    i3 = 0
    for i in range(chicago_res_mean.shape[1]):
        if (chicago_names[i] in chicago_type0):
            i1+=1
            w1 = chicago_res_5[k, i]/chicago_mean[k]
            q1 = chicago_res_25[k, i]/chicago_mean[k]
            q3 = chicago_res_75[k, i]/chicago_mean[k]
            w3 = chicago_res_95[k, i]/chicago_mean[k]
            mean = chicago_res_mean[k, i]/chicago_mean[k]
            med = chicago_res_median[k, i]/chicago_mean[k]
#             if((chicago_names[i]=='TMAX' or chicago_names[i]=='TMIN') and k==2):
#                 w1-=0.15
#                 q1-=0.15
#                 q3-=0.15
#                 w3-=0.15
#                 mean-=0.15
#                 med-=0.15
                
            boxes.append({
                'label' : chicago_names[i] if k==1 else '',
                'name': chicago_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i1%2!=0):
                axs[1][0].add_patch(
                   mpatches.Rectangle((xlims[0][0],i1-.5), xlims[0][1]-xlims[0][0],1, color='lightgrey')
                )
            if(chicago_names[i]=='L10'):
                days.append(229)
            else:
                days.append(round(943/chicago_multiplier[i]))
        elif(chicago_names[i].startswith("L")):
            i3+=1
            w1 = chicago_res_5[k, i]/chicago_mean[k]
            q1 = chicago_res_25[k, i]/chicago_mean[k]
            q3 = chicago_res_75[k, i]/chicago_mean[k]
            w3 = chicago_res_95[k, i]/chicago_mean[k]
            mean = chicago_res_mean[k, i]/chicago_mean[k]
            med = chicago_res_median[k, i]/chicago_mean[k]
            if(chicago_names[i]=='L5' and k==2):
                print(mean)
                w1-=0.02
                q1-=0.02
                q3-=0.02
                w3-=0.02
                mean-=0.02
                med-=0.02
                w1/=2
                q1/=2
                q3/=2
                w3/=2
                mean/=2
                med/=2
                w1+=0.01
                q1+=0.01
                q3+=0.01
                w3+=0.01
                mean+=0.01
                med+=0.01
                
            boxes3.append({
                'label' : chicago_names[i] if k==1 else '',
                'name': chicago_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i3%2!=0):
                axs[1][2].add_patch(
                   mpatches.Rectangle((xlims[2][0],i3-.5), xlims[2][1]-xlims[2][0],1, color='lightgrey')
                )
            days3.append(round(1/chicago_slope2[int(chicago_names[i][1:])-1]))
        else:
            i2+=1
            w1 = chicago_res_5[k, i]/chicago_mean[k]
            q1 = chicago_res_25[k, i]/chicago_mean[k]
            q3 = chicago_res_75[k, i]/chicago_mean[k]
            w3 = chicago_res_95[k, i]/chicago_mean[k]
            mean = chicago_res_mean[k, i]/chicago_mean[k]
            med = chicago_res_median[k, i]/chicago_mean[k]
            boxes2.append({
                'label' : chicago_names[i] if k==1 else '',
                'name': chicago_names[i],
                'whislo': w1*100,    # Bottom whisker position
                'q1'    : q1*100,    # First quartile (25th percentile)
                'med'   : med*100,
                'mean'   : mean*100,    # mean         (50th percentile)
                'q3'    : q3*100,    # Third quartile (75th percentile)
                'whishi': w3*100,    # Top whisker position
                'fliers': [],        # Outliers
            })
            if(i2%2!=0):
                axs[1][1].add_patch(
                   mpatches.Rectangle((xlims[1][0],i2-.5), xlims[1][1]-xlims[1][0],1, color='lightgrey')
                )
            days2.append(round(943/chicago_multiplier[i]))
    
    
    boxes = sort_box(boxes, chicago_type0, other_labels, days)
    boxes2 = sort_box(boxes2, chicago_type1, chicago_labels, days2)
    boxes3 = sort_box(boxes3, chicago_type2, chicago_labels, days3)
    
    x = np.arange(len(boxes)) + (k-1)*0.3
    axs[1][0].bxp(boxes,widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[1][0].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[1][0].set_ylim([-.45,len(boxes)-1+0.45])
    axs[1][0].invert_yaxis()
    
    x = np.arange(len(boxes2)) + (k-1)*0.3
    axs[1][1].bxp(boxes2, widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[1][1].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[1][1].set_ylim([-.45,len(boxes2)-1+0.45])
    axs[1][1].invert_yaxis()
           
    x = np.arange(len(boxes3)) + (k-1)*0.3
    axs[1][2].bxp(boxes3, widths=0.3, positions=x,  showmeans=True, showfliers=False,  vert=False, boxprops=dict(color=colors[k]), capprops = dict(color= colors[k]),\
                 medianprops = dict(color = 'C3') , whiskerprops=dict(color=colors[k]), meanprops = dict(marker = 'x', alpha=0.5))
    axs[1][2].vlines(x=0,ymin=-.45,ymax=9.45,color='grey', ls='--')
    axs[1][2].set_ylim([-.45,len(boxes3)-1+0.45])
    axs[1][2].invert_yaxis()


axs[0][0].set_xlim(xlims[0])
axs[0][1].set_xlim(xlims[1])
axs[0][2].set_xlim(xlims[2])
axs[1][0].set_xlim(xlims[0])
axs[1][1].set_xlim(xlims[1])
axs[1][2].set_xlim(xlims[2])

legend_elements = [ Patch(facecolor='w', edgecolor='C0',
                         label='FHV'),
                    Patch(facecolor='w', edgecolor='C1',
                         label='Taxi'),
                   Patch(facecolor='w', edgecolor='C2',
                         label='BSS')]
axs[0][2].legend(handles = legend_elements, loc='center left', bbox_to_anchor=(1, 0.4),ncol = 1)

axs[0][0].set_title("$x_{weather}$ & $x_{case}$ & $x_{vac}$")
axs[0][1].set_title("$x_{start}$ & $x_{end}$")
axs[0][2].set_title("$x_{linear}$")

axs[1][0].set_xlabel("Trips (%)")
axs[1][1].set_xlabel("Trips (%)")
axs[1][2].set_xlabel("Trips (%/day)")

axs[0][0].set_ylabel("NYC", fontsize=16)
axs[1][0].set_ylabel("Chicago", fontsize=16)


for i in range(3):
    axs[0][i].tick_params(axis='y', which=u'both',length=0)
    axs[1][i].tick_params(axis='y', which=u'both',length=0)


# axs[1][0].text(x = 40, y = 3.5, s='//')
# axs[1][0].set_xticks(np.round(np.arange(-50,101,25)))
# labels = np.arange(-50,101,25)
# labels[-3] = 60
# labels[-2] = 85
# labels[-1] = 110
# axs[1][0].set_xticklabels(labels)

axs[1][2].text(x = 0.7, y = 7.5, s='//')
axs[1][2].set_xticks(np.arange(-1,1.6,0.5))
labels = np.arange(-1,1.6,0.5)
labels[-2] = 2
labels[-1] = 3
axs[1][2].set_xticklabels(labels)
    
plt.tight_layout()
plt.savefig('fig/policy_agg.png', dpi = 300, format = 'png')
plt.savefig('fig/policy_agg.pdf', dpi = 300, format = 'pdf')
# plt.show()

# Spatial impact

In [ ]:
import geopandas as gpd
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry.point import Point
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import TwoSlopeNorm
import matplotlib.ticker as ticker

points = gpd.GeoSeries([Point(-73.5, 40.5), Point(-74.5, 40.5)], crs=4326)  # Geographic WGS 84 - degrees
points = points.to_crs(32619) # Projected WGS 84 - meters
distance_meters = points[0].distance(points[1])

In [ ]:
nyc_map = gpd.read_file("data/nyc_community.shp")
chicago_map = gpd.read_file("data/community.shp")
chicago_map['area_num_1'] = chicago_map['area_num_1'].astype('int')
chicago_map = chicago_map.sort_values(by='area_num_1')

In [ ]:
nyc_population = pd.read_csv("data/NYC_population_2020.csv")
chicago_population = pd.read_csv("data/Chicago_population_2020.csv")
nyc_map = pd.merge(left = nyc_map,  right = nyc_population, left_on = 'boro_cd', right_on = 'AreaID')
chicago_map = pd.merge(left = chicago_map,  right = chicago_population, left_on = 'area_num_1', right_on = 'AreaID')
nyc_map['density'] = nyc_population['Population'] / nyc_map['shape_area'] * 1609 * 1609  / 0.3048 / 0.3048/1000
chicago_map['density'] = chicago_population['Population'] / gpd.GeoSeries.to_crs(chicago_map['geometry'], 'EPSG:26916').area * 1609 * 1609/1000

In [ ]:
fig, ax = plt.subplots(figsize= (7,6))

nyc_map.plot(column='density', legend = True, vmax=100, cmap = 'RdBu_r', edgecolor='white', ax=ax, legend_kwds={'label': "1,000 people per square mile",'shrink':0.8})

ax.set_aspect('auto')
ax.axis('off')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("New York City 2020 population density",fontsize=20)
ax.add_artist(ScaleBar(distance_meters/1609, units ='mi', fixed_value=5, fixed_units='mi', dimension='imperial-length'))

plt.savefig('fig/nyc_map.png', dpi = 300, format = 'png')
plt.savefig('fig/nyc_map.pdf', dpi = 300, format = 'pdf')

In [ ]:
fig, ax = plt.subplots(figsize= (7,6))

chicago_map.plot(column='density', legend = True, cmap = 'RdBu_r', vmax=25, edgecolor='white', ax=ax,  legend_kwds={'label': "1,000 people per square mile",'shrink':0.8})

ax.set_aspect('auto')
ax.axis('off')
ax.set_xlim([-88, -87.45])
ax.set_ylim([41.58, 42.1])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Chicago 2020 population density", fontsize=20)
ax.add_artist(ScaleBar(distance_meters/1609, units ='mi', fixed_value=5, fixed_units='mi', dimension='imperial-length'))

plt.savefig('fig/chicago_map.png', dpi = 300, format = 'png')
plt.savefig('fig/chicago_map.pdf', dpi = 300, format = 'pdf')

In [ ]:
nyc_target_order = ['RAIN', 'RAIN_LOG','TEMP', 'CASE', 'VAC'] + ['S'+str(i) for i in range(1,11)]+['L'+str(i) for i in range(1,11)]+['E'+str(i) for i in range(1,11)]
chicago_target_order = ['RAIN', 'RAIN_LOG','TEMP', 'CASE', 'VAC'] + ['S'+str(i) for i in range(1,10)]+ ['L'+str(i) for i in range(1,10)]+ ['E'+str(i) for i in range(1,10)]
# nyc_new_slope = list(1/np.array([347, 943, 442, 230, 475 , 55, 471, 7,  14,  14,  14, 377, 100,  63]))
# chicago_new_slope = list(1/np.array([340, 943, 514, 229, 510,  74, 376, 457,  21, 318, 101,  68, 85]))

def agg_policy(impact, names, orders, multiplier):
    # name to index projection
    ind_map = {}
    res_orders=orders.copy()
    res_flag = [False] * len(orders)
    for name in names:
        if(name.startswith('RAIN')):
            res_flag[orders.index('RAIN')] = True
        elif(name =='TMAX' or name =='TMIN'):
            res_flag[orders.index('TEMP')] = True
        elif(name.startswith('CASE')):
            res_flag[orders.index('CASE')] = True
        elif(name.startswith('VAC')):
            res_flag[orders.index('VAC')] = True
        elif(len(name)<=4 and name[1:]):
            res_flag[orders.index(name)] = True
    res_orders = [orders[i] for i in range(len(orders)) if res_flag[i]]
    i=0
    for name in names:
        if(name.startswith('RAIN')):
            ind_map[i] = res_orders.index('RAIN')
        elif(name =='TMAX' or name =='TMIN'):
            ind_map[i] = res_orders.index('TEMP')
        elif(name.startswith('CASE')):
            ind_map[i] = res_orders.index('CASE')
        elif(name.startswith('VAC')):
            ind_map[i] = res_orders.index('VAC')
        elif(len(name)<=4):
            ind_map[i] = res_orders.index(name)
        else:
            ind_map[i] = -1
        i+=1
    # initialize result
    res_impact = np.zeros((len(res_orders), impact.shape[1]))
    # using for loop to generate new res
    for i in range(len(names)):
        if(ind_map[i]!=-1):
#             print(ind_map)
            res_impact[ ind_map[i], :] += impact[ i, :]*multiplier[i]
    
    return res_impact, res_orders

In [ ]:
nyc_names = pd.read_csv("result/nyc_u_mean.csv").columns[1:]
chicago_names = pd.read_csv("result/chicago_u_mean.csv").columns[1:]

nyc_multiplier = []
chicago_multiplier = []

nyc_multiplier2 = []
chicago_multiplier2 = []

i=0
for name in nyc_names:
    if(name.startswith('L')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[10+ind-1])
    elif(name.startswith('S')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[ind-1])
    elif(name.startswith('E')):
        ind = int(name[1:])
        nyc_multiplier.append(nyc_slope[20+ind-1])
    elif(name.startswith("RAIN")):
        nyc_multiplier.append(1/347)
    elif(name.startswith("TM")):
        nyc_multiplier.append(1/943)
    elif(name.startswith("CASE")):
        nyc_multiplier.append(1/442)
    elif(name.startswith("VAC")):
        nyc_multiplier.append(1/230)
    i+=1

i=0
for name in chicago_names:
    if(name.startswith('L')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[9+ind-1])
    elif(name.startswith('S')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[ind-1])
    elif(name.startswith('E')):
        ind = int(name[1:])
        chicago_multiplier.append(chicago_slope[18+ind-1])
    elif(name.startswith("RAIN")):
        chicago_multiplier.append(1/340)
    elif(name.startswith("TM")):
        chicago_multiplier.append(1/943)
    elif(name.startswith("CASE")):
        chicago_multiplier.append(1/514)
    elif(name.startswith("VAC")):
        chicago_multiplier.append(1/229)
    else:
        chicago_multiplier.append(1)
    i+=1
        
nyc_multiplier = np.array(nyc_multiplier)*943
chicago_multiplier = np.array(chicago_multiplier)*943

In [ ]:
nyc_num = 142
chicago_num = 154
nyc_v = pd.read_csv("result/nyc_v.csv").values[:,1:]
chicago_v = pd.read_csv("result/chicago_v.csv").values[:,1:]

# contrib, row: factors + signals, col: mean impact, 25 percentile and 75 percentile
chicago_u_mean = pd.read_csv("result/chicago_u_mean.csv").fillna(0)
nyc_u_mean = pd.read_csv("result/nyc_u_mean.csv").fillna(0)
chicago_u_mean['VAC'] += chicago_u_mean['t.data.frame.colMeans.t.t.as.matrix.chicago.df_....chicago_policy.....']

nyc_contrib = nyc_u_mean.values[:,1:]
chicago_contrib = chicago_u_mean.values[:,1:]

nyc_impact = nyc_contrib.T @ nyc_v.T
chicago_impact = chicago_contrib.T @ chicago_v.T
nyc_impact = np.nan_to_num(nyc_impact.astype('float'))
chicago_impact = np.nan_to_num(chicago_impact.astype('float'))
nyc_impact, nyc_new_order = agg_policy(nyc_impact, nyc_names, nyc_target_order, nyc_multiplier)
chicago_impact, chicago_new_order = agg_policy(chicago_impact, chicago_names, chicago_target_order, chicago_multiplier)

# Add vaccine impact
nyc_impact[3,:] += nyc_impact[-1,:]
chicago_impact[3,:] += chicago_impact[-1,:]

nyc_res_mean = []
chicago_res_mean = []
for h in range(3):
    nyc_res_mean.append(nyc_impact[:,nyc_num*h:(nyc_num*(h+1))])
    chicago_res_mean.append(chicago_impact[:,chicago_num*h:(chicago_num*(h+1))])

nyc_res_mean = np.stack(nyc_res_mean)
chicago_res_mean = np.stack(chicago_res_mean)

In [ ]:
nyc_fhv_mean, nyc_taxi_mean, nyc_bike_mean = 9842.222, 3440.607, 747.0655
chicago_fhv_mean, chicago_taxi_mean, chicago_bike_mean = 3712.273, 534.7464, 133.4529

nyc_mean = [nyc_fhv_mean, nyc_taxi_mean, nyc_bike_mean]
chicago_mean = [chicago_fhv_mean, chicago_taxi_mean, chicago_bike_mean]

In [ ]:
def fmt(x, pos):
    a, b = '{:.0e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [ ]:
my_dict = {'S': '$x_{start}$', 'L': '$x_{linear}$', 'E': '$x_{end}$'}

In [ ]:
nyc_new_labels = nyc_new_order[:3] + [nyc_labels[int(i[1:])-1] + ' ' + my_dict[i[0]]  for i in nyc_new_order[3:]]
chicago_new_labels = chicago_new_order[:4] + [chicago_labels[int(i[1:])-1] + ' ' + my_dict[i[0]]  for i in chicago_new_order[4:]]

In [ ]:
# Type 1
fig, axs = plt.subplots(1,6, figsize = (14, 2), sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

to_plot_index = [nyc_new_labels.index("PH1 $x_{linear}$"), nyc_new_labels.index("RO4 $x_{linear}$")]
to_divide = [475, 471]

for i in range(3):
    for j in range(len(to_plot_index)):
        to_plot = nyc_res_mean[i,to_plot_index[j],:]/100
        nyc_map['policy'] = to_plot[:71] #/np.sum(np.abs(to_plot[:71]))
        mymin = min(-5,np.min(to_plot[:71]))
        mymax = max(5,np.max(to_plot[:71]))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[3*j+i])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        nyc_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[i+j*3], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        axs[3*j+i].set_aspect('auto')
        axs[3*j+i].set_xlim([-74.05, -73.7])
        axs[3*j+i].set_ylim([40.55, 40.92])
        axs[3*j+i].set_xticks([])
        axs[3*j+i].set_yticks([])
        
axs[1-1].set_title("PH1 FHV")
axs[2-1].set_title("PH1 Taxi")
axs[3-1].set_title("PH1 BSS")
axs[4-1].set_title("RO4 FHV")
axs[5-1].set_title("RO4 Taxi")
axs[6-1].set_title("RO4 BSS")

axs[0].set_ylabel("NYC $x_{linear}$", fontsize=16)


plt.tight_layout()

# plt.show()
plt.savefig('fig/select_policy_spaitial_nyc.png', dpi = 300, format = 'png')
plt.savefig('fig/select_policy_spaitial_nyc.pdf', dpi = 300, format = 'pdf')

In [ ]:
# Type 1
fig, axs = plt.subplots(1,6, figsize = (14, 2), sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

to_plot_index = [chicago_new_labels.index("BR1 $x_{linear}$"), chicago_new_labels.index("RO2 $x_{linear}$")]
to_divide = [475, 471]

for i in range(3):
    for j in range(len(to_plot_index)):
        to_plot = chicago_res_mean[i,to_plot_index[j],:]/100
        chicago_map['policy'] = to_plot[:77] 
        mymin = min(-5,np.min(to_plot[:77]))
        mymax = max(5,np.max(to_plot[:77]))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[3*j+i])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        chicago_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[i+j*3], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        axs[3*j+i].set_aspect('auto')
        axs[3*j+i].set_xticks([])
        axs[3*j+i].set_yticks([])
        
axs[1-1].set_title("BR1 FHV")
axs[2-1].set_title("BR1 Taxi")
axs[3-1].set_title("BR1 BSS")
axs[4-1].set_title("RO2 FHV")
axs[5-1].set_title("RO2 Taxi")
axs[6-1].set_title("RO2 BSS")

axs[0].set_ylabel("Chicago $x_{linear}$", fontsize=16)


plt.tight_layout()

# plt.show()
plt.savefig('fig/select_policy_spaitial_chicago.png', dpi = 300, format = 'png')
plt.savefig('fig/select_policy_spaitial_chicago.pdf', dpi = 300, format = 'pdf')

In [ ]:
nyc_dict2 = {'CBD':[101,102,103,104,105,106], '<=5mi to CBD':[],'>5mi to CBD':[], 'Airports':[480,483]}
chicago_dict2 = {'CBD':[8,32], '<=5mi to CBD':[], '>5mi to CBD':[], 'Airports':[76]}

# get the centroid of CBD
nyc_cbd_centroid = nyc_map[nyc_map['AreaID'].isin(nyc_dict2['CBD'])].dissolve().centroid
chicago_cbd_centroid = chicago_map[chicago_map['AreaID'].isin(chicago_dict2['CBD'])].dissolve().centroid

nyc_map['distance'] = nyc_map.geometry.apply(lambda row: haversine((row.centroid.y, row.centroid.x),(nyc_cbd_centroid.y[0], nyc_cbd_centroid.x[0]))/1.609)
chicago_map['distance'] = chicago_map.geometry.apply(lambda row: haversine((row.centroid.y, row.centroid.x),(chicago_cbd_centroid.y[0], chicago_cbd_centroid.x[0]))/1.609)
# get the dict
nyc_dict3 = {}
chicago_dict3 = {}

for key, values in nyc_dict2.items():
    new_values = []
    for value in values:
        new_values.append(nyc_map['AreaID'].tolist().index(value))
    nyc_dict3[key]=new_values
    
for key, values in chicago_dict2.items():
    new_values = []
    for value in values:
        new_values.append(chicago_map['area_num_1'].tolist().index(value))
    chicago_dict3[key]=new_values
    
for i in range(len(nyc_map)):
    if(i in nyc_dict3['CBD'] or i in nyc_dict3['Airports']):
        pass
    elif(nyc_map.iloc[i]['distance']<=5):
        nyc_dict3['<=5mi to CBD'].append(i)
    else:
        nyc_dict3['>5mi to CBD'].append(i)
for i in range(len(chicago_map)):
    if(i in chicago_dict3['CBD'] or i in chicago_dict3['Airports']):
        pass
    elif(chicago_map.iloc[i]['distance']<=5):
        chicago_dict3['<=5mi to CBD'].append(i)
    else:
        chicago_dict3['>5mi to CBD'].append(i)

In [ ]:
nyc_select_bool = (np.sign(np.sum(np.abs(((np.sum(np.abs(nyc_res_mean[:,:,:71]), axis= 2)/np.array(nyc_mean)[:,None])/71))>0.05, axis = 0))[2:]).tolist()
nyc_select_ind = [i for i in range(len(nyc_select_bool)) if nyc_select_bool[i] == 1]

In [ ]:
chicago_select_bool = (np.sign(np.sum(np.abs(((np.sum(np.abs(chicago_res_mean[:,:,:77]), axis= 2)/np.array(chicago_mean)[:,None])/77))>0.05, axis = 0))[2:]).tolist()
chicago_select_ind = [i for i in range(len(chicago_select_bool)) if chicago_select_bool[i] == 1]

In [ ]:
# Extract data for bar chart, use a log scale
# step one, transfer dict to index in data matrix
# step two, load all data
nyc_res_selected = np.zeros((nyc_res_mean.shape[0],nyc_res_mean.shape[1]-2, len(nyc_dict2) ))
chicago_res_selected = np.zeros((chicago_res_mean.shape[0],chicago_res_mean.shape[1]-2, len(chicago_dict2)))
for i in range(nyc_res_mean.shape[0]):
    skip=2
    for j in range(nyc_res_mean.shape[1]):
        if(skip):
            skip-=1
            continue
        k = 0
        for _, value in nyc_dict3.items():
            nyc_res_selected[i,j-2,k] = np.sum(nyc_res_mean[i,j,value])
            k += 1
for i in range(chicago_res_mean.shape[0]):
    skip=2
    for j in range(chicago_res_mean.shape[1]):
        if(skip):
            skip-=1
            continue
        k = 0
        for _, value in chicago_dict3.items():
            chicago_res_selected[i,j-2,k] = np.sum(chicago_res_mean[i,j,value])
            k += 1

In [ ]:
# step three, create the figures
fig, axs = plt.subplots(3, 1, figsize = (10, 2.4), sharey = True)
fig.subplots_adjust(hspace=0, wspace=0)

colors = ['C'+str(i) for i in range(4)]
for i in range(nyc_res_selected.shape[2]):
    for j in range(nyc_res_selected.shape[0]):
        axs[j].bar(np.arange(len(nyc_select_ind))+i*0.15-0.3, nyc_res_selected[j,nyc_select_ind,i], width=0.15, color = colors[i], label= list(nyc_dict2.keys())[i])

axs[0].set_yscale('symlog',linthresh=100)
axs[0].set_yticks([-10**2, -10**4, 10**2, 10**4])
axs[1].set_yticks([-10**2, -10**4, 10**2, 10**4])
axs[2].set_yticks([-10**2, -10**4, 10**2, 10**4])

for i in range(3):
    axs[i].set_xlim([-.5, 11.3])
    rect0 = patches.Rectangle((-0.6, -2e5), 1, 5e5, alpha=0.1, facecolor='grey', edgecolor='grey')
    rect = patches.Rectangle((.4, -2e5), 3, 5e5, alpha=0.1, facecolor='C1', edgecolor='grey')
    rect2 = patches.Rectangle((3.4, -2e5), 7, 5e5, alpha=0.1, facecolor='C2', edgecolor='grey')
    rect3 = patches.Rectangle((10.4, -2e5), 1, 5e5, alpha=0.1, facecolor='C0', edgecolor='grey')
    axs[i].add_patch(rect0)
    axs[i].add_patch(rect)
    axs[i].add_patch(rect2)
    axs[i].add_patch(rect3)
    if(i<2):
        axs[i].set_xticks([])
    axs[i].tick_params(axis='x', which='both',length=0)
    axs[i].hlines(y=0, xmin = -.5, xmax = 11.3, linestyles='dashed', color = 'grey')


axs[0].set_ylabel("FHV",fontsize=16)
axs[1].set_ylabel("NYC trips\n Taxi",fontsize=16)
axs[2].set_ylabel("BSS",fontsize=16)
axs[2].set_xticks(list(range(12)))#+[1.8,5.8,8.8])
axs[2].set_xticklabels([nyc_new_labels[i+2][:4].strip() for i in nyc_select_ind])#+['\n $x_{start}$', '\n $x_{linear}$', '\n $x_{end}$'])

# axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.4), ncol=4)
# plt.tight_layout()
plt.savefig('fig/direction_spaitial_nyc.png', dpi = 300, format = 'png')
plt.savefig('fig/direction_spaitial_nyc.pdf', dpi = 300, format = 'pdf')

In [ ]:
# step three, create the figures
fig, axs = plt.subplots(3, 1, figsize = (10, 2.4), sharey = True)
fig.subplots_adjust(hspace=0, wspace=0)

colors = ['C'+str(i) for i in range(4)]
for i in range(chicago_res_selected.shape[2]):
    for j in range(chicago_res_selected.shape[0]):
        axs[j].bar(np.arange(len(chicago_select_ind))+i*0.15-0.3, chicago_res_selected[j,chicago_select_ind,i], width=0.15, color = colors[i], label= list(chicago_dict2.keys())[i])

axs[0].set_yscale('symlog',linthresh=100)
axs[1].set_yscale('symlog',linthresh=100)
axs[2].set_yscale('symlog',linthresh=100)
axs[0].set_yticks([-10**2, -10**4, 10**2, 10**4])
axs[1].set_yticks([-10**2, -10**4, 10**2, 10**4])
axs[2].set_yticks([-10**2, -10**4, 10**2, 10**4])

for i in range(3):
    axs[i].set_xlim([-.5, 13.3])
    rect0 = patches.Rectangle((-0.6, -2e5), 1, 5e5, alpha=0.1, facecolor='grey', edgecolor='grey')
    rect = patches.Rectangle((0.4, -2e5), 4, 5e5, alpha=0.1, facecolor='C1', edgecolor='grey')
    rect2 = patches.Rectangle((4.4, -2e5), 8, 5e5, alpha=0.1, facecolor='C2', edgecolor='grey')
    rect3 = patches.Rectangle((12.4, -2e5), 1, 5e5, alpha=0.1, facecolor='C0', edgecolor = 'grey')
    axs[i].add_patch(rect0)
    axs[i].add_patch(rect)
    axs[i].add_patch(rect2)
    axs[i].add_patch(rect3)
    axs[i].tick_params(axis='x', which='both',length=0)
    axs[i].hlines(y=0, xmin = -.5, xmax = 13.3, linestyles='dashed', color = 'grey')
    
    
axs[0].set_ylabel("FHV",fontsize=16)
axs[1].set_ylabel("Chicago trips\n Taxi",fontsize=16)
axs[2].set_ylabel("BSS",fontsize=16)
axs[2].set_xticks(list(range(14)))#+[1.8,6.8,9.8])
axs[2].set_xticklabels([chicago_new_labels[i+2][:4].strip() for i in chicago_select_ind])#+['\n $x_{start}$', '\n $x_{linear}$', '\n $x_{end}$'])
# axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.4))
# axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.4), ncol=4)
# plt.tight_layout()

plt.savefig('fig/direction_spaitial_chicago.png', dpi = 300, format = 'png')
plt.savefig('fig/direction_spaitial_chicago.pdf', dpi = 300, format = 'pdf')

In [ ]:
# Appendix
fig, axs = plt.subplots(len(nyc_select_ind),6, figsize = (14, 1.75*(len(nyc_select_ind))), sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

to_plot_index =[2+i for i in nyc_select_ind]
to_divide = [475, 471]

for i in range(3):
    for j in range(len(to_plot_index)):
        to_plot = nyc_res_mean[i,to_plot_index[j],:]/100
        nyc_map['policy'] = to_plot[:71] #/np.sum(np.abs(to_plot[:71]))
        mymin = min(-5,min(np.min(to_plot[:71]),np.min(to_plot[71:142])))
        mymax = max(5,max(np.max(to_plot[:71]),np.max(to_plot[71:142])))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[j][2*i])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        nyc_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[j][2*i], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        nyc_map['policy'] = to_plot[71:142] #/np.sum(np.abs(to_plot[:71]))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[j][2*i+1])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        nyc_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[j][2*i+1], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        axs[j][2*i].set_aspect('auto')
        axs[j][2*i].set_xlim([-74.05, -73.7])
        axs[j][2*i].set_ylim([40.55, 40.92])
        axs[j][2*i].set_xticks([])
        axs[j][2*i].set_yticks([])
        
        axs[j][2*i+1].set_aspect('auto')
        axs[j][2*i+1].set_xlim([-74.05, -73.7])
        axs[j][2*i+1].set_ylim([40.55, 40.92])
        axs[j][2*i+1].set_xticks([])
        axs[j][2*i+1].set_yticks([])

axs[0][0].set_title("FHV pickup")
axs[0][2].set_title("Taxi pickup")
axs[0][4].set_title("BSS departure")
axs[0][1].set_title("FHV drop-off")
axs[0][3].set_title("Taxi drop-off")
axs[0][5].set_title("BSS arrival")

for i in range(len(nyc_select_ind)):
    axs[i][0].set_ylabel(nyc_new_labels[nyc_select_ind[i]+2], fontsize=16)


plt.tight_layout()

# plt.show()
plt.savefig('fig/policy_spaitial_nyc.png', dpi = 300, format = 'png')
plt.savefig('fig/policy_spaitial_nyc.pdf', dpi = 300, format = 'pdf')

In [ ]:
# Type 1
fig, axs = plt.subplots(len(chicago_select_ind),6, figsize = (14, 1.75*(len(chicago_select_ind))),  sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

to_plot_index =[2+i for i in chicago_select_ind]

for i in range(3):
    for j in range(len(to_plot_index)):
        to_plot = chicago_res_mean[i,to_plot_index[j],:]/100
        chicago_map['policy'] = to_plot[:77] #/np.sum(np.abs(to_plot[:71]))
        mymin = min(-5,min(np.min(to_plot[:77]),np.min(to_plot[77:154])))
        mymax = max(5,max(np.max(to_plot[:77]),np.max(to_plot[77:154])))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[j][2*i])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        chicago_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[j][2*i], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        chicago_map['policy'] = to_plot[77:154] #/np.sum(np.abs(to_plot[:71]))
        norm = TwoSlopeNorm(vmin=mymin, vcenter=0, vmax=mymax)
        divider = make_axes_locatable(axs[j][2*i+1])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cax.text(0,mymin+1.05*(mymax-mymin),s=r'$\times 100$')
        chicago_map.plot(column = 'policy', cmap = plt.cm.RdBu, legend=True, ax = axs[j][2*i+1], norm = norm, edgecolor = 'lightgrey', cax = cax,
                    legend_kwds={'format':"%.0f"})
        
        axs[j][2*i].set_aspect('auto')
        axs[j][2*i].set_xticks([])
        axs[j][2*i].set_yticks([])
        
        axs[j][2*i+1].set_aspect('auto')
        axs[j][2*i+1].set_xticks([])
        axs[j][2*i+1].set_yticks([])

axs[0][0].set_title("FHV pickup")
axs[0][2].set_title("Taxi pickup")
axs[0][4].set_title("BSS departure")
axs[0][1].set_title("FHV drop-off")
axs[0][3].set_title("Taxi drop-off")
axs[0][5].set_title("BSS arrival")

for i in range(len(chicago_select_ind)):
    axs[i][0].set_ylabel(chicago_new_labels[chicago_select_ind[i]+2], fontsize=16)


plt.tight_layout()

# plt.show()
plt.savefig('fig/policy_spaitial_chicago.png', dpi = 300, format = 'png')
plt.savefig('fig/policy_spaitial_chicago.pdf', dpi = 300, format = 'pdf')

In [ ]:
nyc_impact_fhv_p = nyc_impact[[2+i for i in nyc_select_ind],:142]
nyc_impact_taxi_p = nyc_impact[[2+i for i in nyc_select_ind],142:284]
nyc_impact_bike_p = nyc_impact[[2+i for i in nyc_select_ind],284:426]

nyc_impact2 = np.concatenate([nyc_impact_fhv_p,nyc_impact_taxi_p,
                      nyc_impact_bike_p])

chicago_impact_fhv_p = chicago_impact[[2+i for i in chicago_select_ind],:154]
chicago_impact_taxi_p = chicago_impact[[2+i for i in chicago_select_ind],154:308]
chicago_impact_bike_p = chicago_impact[[2+i for i in chicago_select_ind],308:462]

chicago_impact2 = np.concatenate([chicago_impact_fhv_p, chicago_impact_taxi_p,
                           chicago_impact_bike_p])

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (1.8, 5),  sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

cor = np.corrcoef(nyc_impact2)
name = [nyc_new_order[2:][i] for i in nyc_select_ind]
label = [nyc_new_labels[i+2][:3] for i in nyc_select_ind]

res = []
for i in range(3):
    for j in range(i+1, 3):
        m  = len(label)
        res.append(np.diag(cor[(m*i):(m*(i+1)),:][:,(m*j):(m*(j+1))]))

corr = np.stack(res)
sns.heatmap(corr.T,  cmap='RdBu', vmax=1,vmin=-1, center=0,
            linewidths=.5, cbar = False, cbar_kws={"shrink": .8}, ax=ax)

ax.set_xticks(np.arange(.5,3.5,1))
ax.set_xticklabels(['FHV v.s. Taxi', 'FHV v.s. BSS', 'Taxi v.s. BSS'], rotation= 60,fontsize=14)

ax.set_yticks(np.arange(.5,12.5,1))
ax.set_yticklabels(label, rotation= 0, fontsize=14)

ax.hlines(y = [1,4,11], xmin=0, xmax=3, color='red')

    # ax.text(15, 15, 'Type 1',fontsize= 14)
ax.set_title("NYC", fontsize=18)

plt.tight_layout()

plt.savefig('fig/policy_corr_nyc2.png', dpi = 300, format = 'png')
plt.savefig('fig/policy_corr_nyc2.pdf', dpi = 300, format = 'pdf')

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (2.4, 5),  sharex = False,sharey = False)
fig.subplots_adjust(hspace=0, wspace=0)

cor = np.corrcoef(chicago_impact2)
name = [chicago_new_order[2:][i] for i in chicago_select_ind]
label = [chicago_new_labels[i+2][:3] for i in chicago_select_ind]


res = []
for i in range(3):
    for j in range(i+1, 3):
        m  = len(label)
        res.append(np.diag(cor[(m*i):(m*(i+1)),:][:,(m*j):(m*(j+1))]))

corr = np.stack(res)
sns.heatmap(corr.T,  cmap='RdBu', vmax=1,vmin=-1, center=0
            , linewidths=.5, cbar = True, cbar_kws={"shrink": .8}, ax=ax)

ax.set_xticks(np.arange(.5,3.5,1))
ax.set_xticklabels(['FHV v.s. Taxi', 'FHV v.s. BSS', 'Taxi v.s. BSS'], rotation= 60,fontsize=14)

ax.set_yticks(np.arange(.5,14.5,1))
ax.set_yticklabels(label, rotation= 0, fontsize=14)

ax.set_title("Chicago",fontsize=18)

    # ax.text(15, 15, 'Type 1',fontsize= 14)
    
ax.hlines(y = [1,5,13], xmin=0, xmax=3, color='red')

plt.tight_layout()

plt.savefig('fig/policy_corr_chicago2.png', dpi = 300, format = 'png')
plt.savefig('fig/policy_corr_chicago2.pdf', dpi = 300, format = 'pdf')